**MySQL Setup on Colab**

In [1]:
# (source: https://stackoverflow.com/questions/62065617/how-to-read-write-local-mysql-server-8-from-google-colab-with-pyspark/62081837#62081837) 

# install, set connection

!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.autopandas = True
%sql mysql+pymysql://root:root@/
# query using %sql or %%sql
df = %sql SELECT Host, User, authentication_string FROM mysql.user
df

 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.
     |████████████████████████████████| 43 kB 1.6 MB/s 
 * mysql+pymysql://root:***@/


,Host,User,authentication_string
0,localhost,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B
1,localhost,mysql.session,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
2,localhost,mysql.sys,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
3,localhost,debian-sys-maint,*2A2021CADC0C2036FB1614B880869B2AFCF705E7


**MySQL Create Database**

In [2]:

# Create a Database & check if exists
%sql CREATE DATABASE colabdb
%sql SHOW DATABASES

 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/


,Database
0,information_schema
1,colabdb
2,mysql
3,performance_schema
4,sys


**MySQL Create Table**

In [3]:
# Create a table & check if it exists

%sql USE colabdb
x = %sql CREATE TABLE customers (name VARCHAR(255), address VARCHAR(255))
y = %sql CREATE TABLE extratable (name VARCHAR(255), address VARCHAR(255))#, phone INT)

%sql SHOW TABLES

 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/


,Tables_in_colabdb
0,customers
1,extratable


**MySQL Primary Key**

In [4]:
# Primary key: add a column of unique 
# identifiers to table
# for each row, integer from 1
# and increasing (2, 3, 4, ...)

a = %sql CREATE TABLE customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))

#Table already exists, so alter it instead

b = %sql ALTER TABLE customers ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY

 * mysql+pymysql://root:***@/
(pymysql.err.OperationalError) (1050, "Table 'customers' already exists")
[SQL: CREATE TABLE customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
 * mysql+pymysql://root:***@/


**Insert into table**

In [5]:
%sql INSERT INTO customers (name, address) VALUES ('Tom', '18, Busy St.');
%sql INSERT INTO customers (name, address) VALUES ('Jason', '21, Main St.');
%sql INSERT INTO customers (name, address) VALUES ('Alfred', '12, Maple Av.')
%sql SHOW COLUMNS FROM customers
# %sql DELETE * IN customers

# Show table content:
%sql SELECT * FROM customers

 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/


,name,address,id
0,Tom,"18, Busy St.",1
1,Jason,"21, Main St.",2
2,Alfred,"12, Maple Av.",3


**MySQL Select From**

In [6]:
# To select all records of a table:
%sql SELECT * FROM customers

 * mysql+pymysql://root:***@/


,name,address,id
0,Tom,"18, Busy St.",1
1,Jason,"21, Main St.",2
2,Alfred,"12, Maple Av.",3


In [7]:
# Select columns from table:
%sql SELECT name, address FROM customers

 * mysql+pymysql://root:***@/


,name,address
0,Tom,"18, Busy St."
1,Jason,"21, Main St."
2,Alfred,"12, Maple Av."


**MySQL Where:**
Use a filter when selecting records

In [8]:
%sql SELECT * FROM customers WHERE address = '18, Busy St.'

 * mysql+pymysql://root:***@/


,name,address,id
0,Tom,"18, Busy St.",1


Wildcard Characters:

In [9]:
# Include wildcard characters to represent unknown characters

%sql SELECT * FROM customers WHERE address LIKE '%st%'

 * mysql+pymysql://root:***@/


,name,address,id
0,Tom,"18, Busy St.",1
1,Jason,"21, Main St.",2


In [10]:
# To prevent SQL Injection:

# Use the placeholder method (%s) to escape query characters provided by the user (when using cursor method)
# mycursor = dbname.cursor()
# sql = "SELECT * FROM customers WHERE name = %s"
# filter = "Jason"
# mycursor.execute(sql, filter)
# result = mycursor.fetchall()

# for x in result:
#  print(x)

**SQL Order By**

In [11]:
# ORDER BY statement sorts the results
# (ascending order by default)
%sql SELECT * FROM customers ORDER BY name

 * mysql+pymysql://root:***@/


,name,address,id
0,Alfred,"12, Maple Av.",3
1,Jason,"21, Main St.",2
2,Tom,"18, Busy St.",1


DESC

In [12]:
# Order in descending order with DESC
%sql SELECT * FROM customers ORDER BY address DESC

 * mysql+pymysql://root:***@/


,name,address,id
0,Jason,"21, Main St.",2
1,Tom,"18, Busy St.",1
2,Alfred,"12, Maple Av.",3


**DELETE FROM**

In [13]:
# Using a WHERE condition specifies which records will be deleted 
# Otherwise all records are deleted

%sql DELETE FROM customers WHERE id = 2; SELECT * FROM customers

 * mysql+pymysql://root:***@/


,name,address,id
0,Tom,"18, Busy St.",1
1,Alfred,"12, Maple Av.",3


In [ ]:
# Prevent SQL injection by using placeholder %s to escape query characters (when using cursor method)

# mycursor = dbname.cursor()

# sql = "DELETE FROM customers WHERE address = %s"
# filter = ("18, Busy St.")

# mycursor.execute(sql, filter)
# dbname.commit()

# print(mycursor.rowcount, "records deleted")

**Drop Table**

In [14]:
# Delete a table
%sql DROP TABLE extratable

# Check if it worked
%sql SHOW TABLES

 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/


,Tables_in_colabdb
0,customers


In [15]:
# To prevent an error where the table does not exist, 
%sql DROP TABLE IF EXISTS extratable

 * mysql+pymysql://root:***@/


""


**UPDATE TABLE**

In [16]:
%sql UPDATE customers SET address = '182 Grand Canyon' WHERE address = '18, Busy St.'
%sql SELECT * FROM customers

# Preventing SQL injection also applies in updating

 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/


,name,address,id
0,Tom,182 Grand Canyon,1
1,Alfred,"12, Maple Av.",3


**LIMIT:** Limit the results of a query

In [17]:
%sql SELECT * FROM customers LIMIT 1

 * mysql+pymysql://root:***@/


,name,address,id
0,Tom,182 Grand Canyon,1


OFFSET: start from a different place (other than the beginning)

In [18]:
%sql SELECT * FROM customers LIMIT 1 OFFSET 1

# This starts the query after the 1st position (at the second entry/row)

 * mysql+pymysql://root:***@/


,name,address,id
0,Alfred,"12, Maple Av.",3


**JOIN**: Join 2 or more tables

Create 2 new tables

In [19]:
%sql CREATE TABLE buyers (name VARCHAR(255), fav INT(255))
%sql CREATE TABLE items (id INT(255), name VARCHAR(255))

# Add items to the tables
%sql INSERT INTO buyers (name, fav) VALUES ('Steve', 12), ('Anthony', 31), ('Matt', 9), ('Terry', 18), ('Ahmed', 0)
%sql INSERT INTO items (id, name) VALUES (3, 'Latte'), (9, 'Espresso'), (12, 'Black Coffee'), (15, 'Cappuccino'), (18, 'Americano'), (31, 'Lungo')

 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/


""


In [20]:
%sql INSERT INTO buyers (name) VALUES ('Victor') /*add one row with an empty 'fav' cell*/

 * mysql+pymysql://root:***@/


""


Preview tables

In [21]:
%sql SELECT * FROM buyers

 * mysql+pymysql://root:***@/


,name,fav
0,Steve,12.0
1,Anthony,31.0
2,Matt,9.0
3,Terry,18.0
4,Ahmed,0.0
5,Victor,NaN


In [22]:
%sql SELECT * FROM items

 * mysql+pymysql://root:***@/


,id,name
0,3,Latte
1,9,Espresso
2,12,Black Coffee
3,15,Cappuccino
4,18,Americano
5,31,Lungo


Join tables using JOIN/INNER JOIN:
This will exclude rows from both tables that don't match (where 'buyers.fav' does not match any cell in 'item.id')

In [23]:
%sql SELECT buyers.name as buyers, items.name as favorite/*, items.id as item_code*/ FROM buyers /*INNER */JOIN items ON buyers.fav = items.id

 * mysql+pymysql://root:***@/


,buyers,favorite
0,Matt,Espresso
1,Steve,Black Coffee
2,Terry,Americano
3,Anthony,Lungo


Left Join: Includes rows from table one that have no match on table 2

In [24]:
%sql SELECT buyers.name AS buyers, items.name as favorite FROM buyers LEFT JOIN items ON buyers.fav = items.id

 * mysql+pymysql://root:***@/


,buyers,favorite
0,Matt,Espresso
1,Steve,Black Coffee
2,Terry,Americano
3,Anthony,Lungo
4,Ahmed,None
5,Victor,None


Right Join: Includes rows from table 2 that have no match on table 1

In [25]:
%sql SELECT buyers.name as buyers, items.name as favorite FROM buyers RIGHT JOIN items ON buyers.fav = items.id

 * mysql+pymysql://root:***@/


,buyers,favorite
0,Steve,Black Coffee
1,Anthony,Lungo
2,Matt,Espresso
3,Terry,Americano
4,None,Latte
5,None,Cappuccino
